**Resumes**

Sorry for downloading your resume. I need it for testing

In [ ]:
resume = "./resumes/cw-resume-jv1.pdf"

In [ ]:
resume = "./resumes/Jerick-Iquin.pdf"

In [ ]:
resume = "./resumes/Kenneth Lee Resume.pdf"

In [ ]:
resume = "./resumes/Justin Lam - Resume - vPortfolio.pdf"

**Using pdfminer.six**

- Probably will use. Best so far

In [ ]:
from pdfminer.high_level import extract_text, extract_pages
from pdfminer.layout import LAParams

laparams = LAParams(line_margin=1)

In [ ]:
# extract pages outputs an iterable so we can create list from it
pages = [x for x in extract_pages(resume)]

for page in range(len(pages)):
    print('page_number', page)
    text = extract_text(resume, page_numbers=page, laparams=laparams)
    print(text)

**Testing Beautiful Soup**

This is to do webscraping of your linkedin profile

- Linkedin detects automated scraping and blocks requests
- Need to install chrome and chromedriver

In [300]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

def scrape_linkedin_profile(url):
    options = Options()
    # running in headless mode
    options.add_argument('--headless') 
    driver = webdriver.Chrome(executable_path='chromedriver', options=options)

    driver.get(url)

    # wait for the page to load. adjust sleep here
    # driver.implicitly_wait(10)
    time.sleep(5)

    html_content = driver.page_source

    soup = BeautifulSoup(html_content, 'html.parser')

    lis = soup.find_all('li', class_='experience-item')

    modal_dismis = driver.find_element(By.CLASS_NAME, 'contextual-sign-in-modal__modal-dismiss')
    if modal_dismis:
        modal_dismis.click()

    show_more_buttons = driver.find_elements(By.CLASS_NAME, 'show-more-less-text__button--more')

    for button in show_more_buttons:
        button.click()

    for li in lis:
        print("Experience", lis.index(li))
        description_more = ''
        description_less = ''
        
        title = get_text(li.find('h3', class_='profile-section-card__title'))
        company = get_text(li.find('h4', class_='profile-section-card__subtitle'))
        location = get_text(li.find('p', class_='experience-item__location'))

        # the class naming is weird in this case. Everything that has show more button will have --more
        # modifier and everything else has --less modifier 
        description_more = get_text(li.find('p', class_='show-more-less-text__text--more'))
        description_less = get_text(li.find('p', class_='show-more-less-text__text--less'))

        if description_more:
            description_list = get_duties(description_more)
        elif description_less:
            description_list = get_duties(description_less)
        else:
            description_list = []
            
        print(title)
        print(company)
        print(location)
        print('Responsibilities: ')
        for duty in description_list:
            print(duty)
        print('\n')
        
    driver.quit()

def get_text(element):
    try:
        return element.get_text().strip()
    except Exception as e:
        return ''
    
def get_duties(description):
    # for some reason the button "show less" keeps popping up. Need to remove that
    trimmed = description.replace('\n            \n\n    \n    \n\n    \n        Show less', '')
    return trimmed.split('• ')[1:]


In [302]:
scrape_linkedin_profile('https://www.linkedin.com/in/jackcpalmer/')

Experience 0
Front End Developer
Expo Vente a Canadá

Responsibilities: 
Implemented 10+ front end messaging features including localized timestamps, URL parsing, emoji support, modals and drawers, leveraging React Router for page routing. Improving chat accessibility by doubling the QoL features for users.
Responsible for server support during a live event. Stayed in frequent contact with the client and provided updates on site health. Utilized AWS and Google analytics to ensure server load was sustainable, Docker containers were healthy and tracked site traffic metrics. Responsible for redeploying of site in the event of a crash.
Reformatted existing front end architecture, removing 16,000+ lines of redundant data, reducing bundle size by more than 25%, resulting in improved responsiveness and performance metrics of over 20%.
Handpicked by instructor from a pool of 50+ graduates to collaborate on the front end tickets written in Typescript and React.


Experience 1
Sports Coach - You

**Using pypdf package**

- Didn't work on Jek's email. Some of the spaces are not being extracted.

In [ ]:
from pypdf import PdfReader

reader = PdfReader(resume)
number_of_pages = len(reader.pages)

for page_number in range(number_of_pages):
    page = reader.pages[page_number]
    text = page.extract_text()
    print(text)

**Using pyPDF2**

- Didn't work on Jek's email. Some of the spaces are not being extracted.

In [ ]:
from PyPDF2 import PdfReader

reader = PdfReader(resume, strict=True)
number_of_pages = len(reader.pages)
for page_number in range(number_of_pages):
    page = reader.pages[page_number]
    text = page.extract_text()
    print(text)
